In [1]:
#| default_exp services.console.base

In [2]:
#| exporti
import json
from typing import List

from dc_logger.client.base import ServiceHandler, ServiceConfig, LogEntry


In [3]:
#| exporti

from dc_logger.client.base import ServiceConfig, OutputMode
from dataclasses import dataclass

In [4]:
#| export

@dataclass
class Console_ServiceConfig(ServiceConfig):
    """Console-specific log configuration"""
    
    output_mode: OutputMode = "console"
    output_type: str = "text"  # "text" or "json"

    def validate_config(self) -> bool:
        if self.output_type not in ["text", "json"]:
            raise ValueError(f"output_type must be 'text' or 'json', got {self.output_type}")
        return True

In [5]:
#| export

class ConsoleHandler(ServiceHandler):
    """Handler for console output"""

    async def _write_json(self, entry: LogEntry) -> dict:
        message = json.dumps(entry.to_dict(), indent=2, default=str)
        print(message)
        return message

    async def _write_text(self, entry: LogEntry) -> str:
        message = f"[{entry.timestamp}] {entry.level.value} - {entry.message}"
        print(message)
        return message
                
    async def write(self, entries: List[LogEntry]) -> bool:
        """Write log entries to console"""
        if not isinstance(entries, list):
            entries = [entries]
        
        # Get output type from config
        output_type = self.service_config.output_type if hasattr(self.service_config, 'output_type') else "text"
        
        try:
            for entry in entries:
                if output_type == "json":
                    await self._write_json(entry)
                else:
                    await self._write_text(entry)
            return True
                
        except Exception as e:
            print(f"Error writing to console: {e}")
            return False

    async def flush(self) -> bool:
        """Console output doesn't need flushing"""
        return True


In [6]:
# | hide
import nbdev;
nbdev.nbdev_export('./base.ipynb')